In [ ]:
from gzip import open # NB: overrides standard open()
import pickle as pkl
import tensorly as tl
from tensorly.base import tensor_to_vec,  partial_tensor_to_vec
import numpy as np
import pandas as pd
import warnings 
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

warnings.simplefilter('ignore') # отключим предупреждения Anaconda
Xdata_numpy = pkl.load(open('C:/Users/admin/Desktop/88/X.pkl.gz', 'rb'))
Ydata = pkl.load(open('C:/Users/admin/Desktop/88/y.pkl.gz', 'rb'))

print(type(Xdata_numpy["X"]))

#делим набор на обучающий и тестовый
X_train, X_test, y_train, y_test = train_test_split(
     Xdata_numpy["X"], Ydata, test_size=0.2857 
)

print( y_train.shape)
# прописываем класс регрессии
from sklearn.base import BaseEstimator
from sklearn.metrics import r2_score
from tensorly.regression.kruskal_regression import KruskalRegressor

class WrappedKruskalRegressor(BaseEstimator):
	def __init__(self, weight_rank = 1, tol = 1e-6, reg_W = 1, n_iter_max = 100):
		self.weight_rank = weight_rank
		self.tol = tol
		self.reg_W = reg_W
		self.n_iter_max = n_iter_max
	
	def fit(self, X, y = None):
		self._model = KruskalRegressor(self.weight_rank, self.tol, self.reg_W, self.n_iter_max)
		self._model.fit(X, y)
		if self._model.n_iterations_ == self.n_iter_max:
			raise Exception('{} did not converge'.format(self))
		return self
	
	def predict(self, X):
		return self._model.predict(X)
	
	def score(self, X, y):
		return r2_score(y, self.predict(X))


#начинаем применять

from sklearn.model_selection import GridSearchCV
regressioModel=WrappedKruskalRegressor()
parametrsNames={'n_iter_max': [30000],
                'reg_W': range(1,21),
                'tol': [1e-04,1e-05,1e-06],
                'weight_rank': range(1,21)
               }

gridCought=GridSearchCV(regressioModel, parametrsNames, cv=5)
gridCought.fit(X_train,y_train.iloc[:,0])
predictors=gridCought.predict(X_test)

print(gridCought.best_params_)

<class 'numpy.ndarray'>
(25, 3)

Converged in 29 iterations

Converged in 525 iterations

Converged in 477 iterations

Converged in 66 iterations

Converged in 60 iterations

Converged in 42 iterations

Converged in 17 iterations

Converged in 19 iterations

Converged in 17 iterations

Converged in 67 iterations

Converged in 130 iterations

Converged in 72 iterations

Converged in 40 iterations

Converged in 89 iterations

Converged in 8 iterations

Converged in 90 iterations

Converged in 39 iterations

Converged in 64 iterations

Converged in 112 iterations

Converged in 15 iterations

Converged in 7 iterations

Converged in 9 iterations

Converged in 20 iterations

Converged in 108 iterations

Converged in 39 iterations

Converged in 77 iterations

Converged in 65 iterations

Converged in 40 iterations

Converged in 120 iterations

Converged in 46 iterations

Converged in 34 iterations

Converged in 16 iterations

Converged in 13 iterations

Converged in 97 iterations

Converged in

In [ ]:
gridCought.best_estimator_

In [ ]:
gridCought.score(X_test, y_test.iloc[:,0])

In [ ]:
plt.plot(y_test.iloc[:,0],y_test.iloc[:,0],color="blue")
plt.plot(y_test.iloc[:,0],predictors,".",color="red")
plt.xlabel("Истинные значения",fontsize=12)
plt.ylabel("Предсказанные значения", fontsize=12)
plt.grid()
plt.show();

In [ ]:
gridCought.scorer_

In [ ]:
gridCought.score

In [ ]:
 gridCought.cv_results_

In [ ]:
gridCought.cv_results_['mean_test_score']

In [ ]:
cvres  = gridCought.cv_results_

In [ ]:
gridCought.cv_results_['mean_test_score'][6]

In [ ]:
min(gridCought.cv_results_['mean_test_score'])

In [ ]:
for mean_score,  params  in zip(cvres [ "mean_test_score" ], 
                                cvres [ "params" ]
                                ): 
    print (np.sqrt(mean_score**2) , params) 

In [ ]:
score=[]
score_l=[]
parametr_reg_w=[]
my_error=[]

In [ ]:
def f(x,y):
    my_error=[]
    for i in range(0,len(y)):
        my_error+=[(x[i]-y[i])**2]
    return my_error
my_error=f(np.array(y_test.iloc[:,0]),predictors)
print(my_error)

In [ ]:
for mean_score,  params  in zip (cvres [ "mean_test_score"],cvres [ "params" ] )  : 
    score+=[np.array(mean_score)**2]
    score_l+=[np.array((mean_score))]
    parametr_reg_w+=[params["reg_W"]]
print (score,"   ",parametr_reg_w) 


In [ ]:
plt.plot(parametr_reg_w,score_l,".",color="red")
#plt.plot(parametr_reg_w,score_l,".",color="blue")
plt.grid()
plt.ylabel("Ошибка",fontsize=15)
plt.xlabel("Параметр",fontsize=15)
plt.title("График зависимости квадрата ошибки от reg_W",fontsize=18)
plt.show();

In [ ]:
?file_obj.write(plt)